# <u>Tut_5.1</u>

## Learning outcomes

* *Medical charges* walkthrough project linear regression (*finished*)
* Course Work
* Homework

---

### Linear regression code-along BDA/Data Science project (finished)

#### Import packages

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px # Interactive charts and save some coding; .express - high-level api
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Change settings to improve default style (optional)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

* Data path

In [ ]:
data_path = r'C:\Users\soyak\OneDrive\Documents\m32895\inputs\raw_data\medical-charges.csv'

* Load data

In [ ]:
medical_df = pd.read_csv(data_path)

In [ ]:
medical_df.head()

#### Correlation

In [ ]:
print(medical_df['charges'].corr(medical_df['age']))
print(medical_df['charges'].corr(medical_df['bmi']))
print(medical_df['charges'].corr(medical_df['children']))

In [ ]:
smoker_values = {'no':0, 'yes':1}
smoker_numeric = medical_df['smoker'].map(smoker_values)
medical_df['charges'].corr(smoker_numeric)

* Correlation **matrix**

In [ ]:
medical_df.select_dtypes(include='number').corr()

* Heatmap

In [ ]:
sns.heatmap(
    medical_df.select_dtypes(include='number').corr(),
    cmap='Reds',
    annot=True    
)
plt.title("Correlation matrix")
plt.show()

<u>Correlation vs. Causation fallacy: hight correlation cannot be used to study causation.</u>

## Linear regression with Scikit-learn

* Import Scikit-learn module

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()  # Create a model variable and assign linear regression model to it

In [ ]:
help(model.fit)

In [ ]:
non_smoker_df = medical_df[medical_df['smoker'] == 'no'] # Either line will work. Just different syntaxes; But I like no 1 more.
non_smoker_df.head(3)

In [ ]:
inputs = non_smoker_df[['age']] # required to be 2D array. We want a dataframe, not a series.
targets = non_smoker_df['charges'] # There is only one dependent variable => no 2D notation is required.
print("Inputs shape", inputs.shape) # Output: (1064, 1) -> First number is how many lines; 2nd number is how many columns.
print("targets", targets.shape)

In [ ]:
model.fit(inputs, targets)

In [ ]:
model.predict(np.array([
    [23],
    [37],
    [61]
]))

In [ ]:
predictions = model.predict(inputs)
print(predictions)

* Let's compute RMSE to evaluate the model

In [ ]:
def rmse(targets, predictions):
    """
    Returns RMSE for targets and prediction values.
    """
    return np.sqrt(np.mean(np.square(predictions - targets)))

In [ ]:
rmse(targets, predictions) # Output USD 4662.5. Meaning on average we are away from the target by this number.

#### Model coefficients

In [ ]:
# w:
print(model.coef_)
# b:
print(model.intercept_)

### Machine Learning

* Three components of any ML problem:
	* Model (Linear? Polynomial?... How do you want to fit?)
	* Cost function
	* Optimizer

#### Machine Learning Workflow

![ML workflow](../assets/img/ml_workflow.jpg)

### Linear regression with multiple features

In [ ]:
# Create inputs and targets:
inputs, targets = non_smoker_df[['age', 'bmi']], non_smoker_df['charges']
# Create and train the model:
model = LinearRegression().fit(inputs, targets)
# Run predictions:
predictions = model.predict(inputs)  # inputs [[22, 20]]
print(f"Predicted charge is: {predictions}")
# Compute loss to evaluate model:
loss = rmse(targets, predictions)
print(f"The loss is: {round(loss, 2)}")

In [ ]:
inputs.head()

In [ ]:
inputs.shape

In [ ]:
non_smoker_df['charges'].corr(non_smoker_df['bmi'])

In [ ]:
fig = px.scatter(
    non_smoker_df,
    x='bmi',
    y='charges',
    title="BMI vs. Charges"
)
fig.update_traces(marker_size=5)
fig.show()

* We can also visualize the relationship on the 3D scatter plot for all 3 variables: 'age', 'bmi', and 'charges'.

In [ ]:
fig = px.scatter_3d(
    non_smoker_df,
    x='age',
    y='bmi',
    z='charges'
)
fig.update_traces(marker_size=3, marker_opacity=0.8)
fig.show()

In [ ]:
print("Model coefficient and intercept are:")
print(model.coef_)
print(round(model.intercept_, 2))

In [ ]:
non_smoker_df['children'].corr(non_smoker_df['charges'])

In [ ]:
fig = px.strip(
    non_smoker_df,
    x='children',
    y='charges',
    title="Children vs. Charges"    
)
fig.update_traces(marker_size=4, marker_opacity=0.7)
fig.show()

* Now: what happens if we ignore the difference between smokers and non-smokers?

In [ ]:
# Create inputs and targets:
inputs, targets = medical_df[['age', 'bmi', 'children']], medical_df['charges']
# Create and train the model:
model = LinearRegression().fit(inputs, targets)
# Run predictions:
predictions = model.predict(inputs)
# compute the loss and evaluate the model:
loss = rmse(targets, predictions)
print(f"The loss is: {round(loss, 2)}")
"""
The output of the above code will be way much higher because smoker/non-smoker feature makes distinct clusters - see plot below.
"""

In [ ]:
px.scatter(
    medical_df,
    x='age',
    y='charges',
    color='smoker',
    title="Charges vs. Age"
)

#### Using categorical features for machine learning

* If we could use categorical columns like "smoker", we can train a single model fof the entire dataset
* To use categorical columns, we simply need to convert them to numbers. There are 3 possible ways:
	* If a categorical column has just two categories (in this case it is called a binary category), we can replace their values with "1" and "0"
	* If a categorical column has more than 2 categories, we can perform one-hot encoding, i.e. create a new column for each category with "1" and "0"
	* If the categories have natural order, e.g. "hot", "warm", "neutral", "cold", we can convert them to numbers 1, 2, 3, 4 preserving the logical order. These are called **ordinals**

### Binary categories

* The "smoker" category has just two values "yes" and "no". Let's create a new column "smoker_code" containing 0 for "no" and 1 for "yes".

In [ ]:
sns.barplot(data=medical_df, x='smoker', y='charges')

In [ ]:
smoker_codes = {'no': 0, 'yes': 1}
medical_df['smoker_code'] = medical_df.smoker.map(smoker_codes)

In [ ]:
medical_df['charges'].corr(medical_df['smoker_code'])

In [ ]:
medical_df

We can now use the `smoker_code` column for linear regression.

$charges = w_1 \times age + w_2 \times bmi + w_3 \times charges + w_4 \times smoker + b$

In [ ]:
# Create inputs and targets
inputs, targets = medical_df[['age', 'bmi', 'children', 'smoker_code']], medical_df['charges']

# Create and train the model
model = LinearRegression().fit(inputs, targets)

# Generate predictions
predictions = model.predict(inputs)

# Compute loss to evalute the model
loss = rmse(targets, predictions)
print('Loss:', loss)

The loss reduces from `11355` to `6056`, by approx. 50%! This is an important lesson: **never ignore categorical data.**


Let's try adding the "sex" column as well.

$charges = w_1 \times age + w_2 \times bmi + w_3 \times charges + w_4 \times smoker + w_5 \times sex + b$

In [ ]:
sns.barplot(data=medical_df, x='sex', y='charges')

In [ ]:
sex_codes = {'female': 0, 'male': 1}
medical_df['sex_code'] = medical_df.sex.map(sex_codes)
medical_df['charges'].corr(medical_df['sex_code'])

In [ ]:
# Create inputs and targets
inputs, targets = medical_df[['age', 'bmi', 'children', 'smoker_code', 'sex_code']], medical_df['charges']

# Create and train the model
model = LinearRegression().fit(inputs, targets)

# Generate predictions
predictions = model.predict(inputs)

# Compute loss to evalute the model
loss = rmse(targets, predictions)
print('Loss:', loss)

* As you might expect, this does not have a significant impact on the loss.

### One-hot Encoding

The "region" column contains 4 values, so we'll need to use hot encoding and create a new column for each region.

![](https://i.imgur.com/n8GuiOO.png)

In [ ]:
sns.barplot(data=medical_df, x='region', y='charges')

In [ ]:
from sklearn import preprocessing


In [ ]:
enc = preprocessing.OneHotEncoder()
enc.fit(medical_df[['region']])
enc.categories_

In [ ]:
one_hot = enc.transform(medical_df[['region']]).toarray()
one_hot

In [ ]:
medical_df[['northeast', 'northwest', 'southeast', 'southwest']] = one_hot

In [ ]:
medical_df.head()

Let's include the region columns into our linear regression model.

$charges = w_1 \times age + w_2 \times bmi + w_3 \times charges + w_4 \times smoker + w_5 \times sex + w_6 \times region + b$

In [ ]:
# Create inputs and targets
input_cols = ['age', 'bmi', 'children', 'smoker_code', 'sex_code', 'northeast', 'northwest', 'southeast', 'southwest']
inputs, targets = medical_df[input_cols], medical_df['charges']

# Create and train the model
model = LinearRegression().fit(inputs, targets)

# Generate predictions
predictions = model.predict(inputs)

# Compute loss to evalute the model
loss = rmse(targets, predictions)
print('Loss:', loss)

* This leads to a fairly small reduction in the loss. 

### Creating a Test Set

* Models like the one we've created in this tutorial are designed to be used in the real world
* It's common practice to set aside a small fraction of the data (e.g. 10%) just for testing and reporting the results of the model.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.1)

In [ ]:
# Create and train the model
model = LinearRegression().fit(inputs_train, targets_train)

# Generate predictions
predictions_test = model.predict(inputs_test)

# Compute loss to evalute the model
loss = rmse(targets_test, predictions_test)
print('Test Loss:', loss)

* Let's compare this with the training loss.

In [ ]:
# Generate predictions
predictions_train = model.predict(inputs_train)

# Compute loss to evalute the model
loss = rmse(targets_train, predictions_train)
print('Training Loss:', loss)

### Let's make a prediction for a single person based on our model

In [ ]:
john_smith = {
	"age": [30],             # Example age
    "bmi": [29.5],           # Example BMI
    "children": [2],         # Example number of children
    "smoker_code": [1],      # Smoker (1 for yes, 0 for no)
    "sex_code": [0],         # Male (1), Female (0)
    "northeast": [0],        # One-hot encoded region:
    "northwest": [1],
    "southeast": [0],
    "southwest": [0]
}


* Create a single-person DataFrame with john_smith data

In [ ]:
single_person = pd.DataFrame(john_smith)
single_person.head()

* ### Make a prediction for a single person

In [ ]:
single_prediction = model.predict(single_person)
print(f"Predicted health insurance charge for John Smith is {single_prediction[0]:.1f} USD")


* ### Prediction interval

In [ ]:
residuals = targets_train - predictions_train  # Compute residuals (errors)
std_dev = np.std(residuals)  # Compute the standard deviation of residuals
confidence_interval = 1.96 * std_dev  # Confidence interval (for 95% confidence level, 1.96 standard deviations)

# Make a single prediction
# single_prediction = model.predict(single_person)[0]

# Compute lower and upper bounds for prediction interval
lower_bound = single_prediction - confidence_interval
upper_bound = single_prediction + confidence_interval

print(f"Predicted value: {single_prediction}")
print(f"95% Prediction Interval: ({lower_bound}, {upper_bound})")


---
---

### Course Work (continued)

---

#### End of tutorial.
You can now `add`, `commit` and `push`

---